In [44]:
import sys
sys.path.append("../../")

In [45]:
from multi_tp.models_ids import *
from multi_tp.utils import get_model_name_path, get_suffix, LANGUAGES, pivot_file_tmpl

In [46]:
# Main hyperparameters
system_role = "normal"
translator_provider_forward = "google"
translator_provider_backward = "google"
analysis_backend_model_version = "meta-llama/Meta-Llama-3.1-8B-Instruct"
add_paraphrase = False
country = None

In [47]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import os 
def get_data(df):
    df_plot = df.copy()
    df_plot = df_plot.rename(columns={"criterion": "lang"})
    df_plot.index=df_plot["lang"]    
    df_plot = df_plot.replace("---", 0)
    if "consistency_by_swapping" in df_plot.index:
        df_plot = df_plot.drop(index="consistency_by_swapping")
    df_plot = df_plot.drop(columns=["lang"])
    df_plot = df_plot.astype(float)
    df_plot = df_plot.T
    df_plot["lang"] = df_plot.index
    return df_plot 

# Load data for all models
all_data = {}
original_data = {}
for model_version in MODELS:
   
    input_path = pivot_file_tmpl.format(
        model_version=get_model_name_path(model_version),
        system_role=system_role,
        suffix=get_suffix(add_paraphrase, country),
        translator_provider_forward=translator_provider_forward,
        translator_provider_backward=translator_provider_backward,
        analysis_backend_model_version=get_model_name_path(analysis_backend_model_version),
    )
    input_path = os.path.join("../../", input_path)
    if not os.path.exists(input_path):
        continue
    df = pd.read_csv(input_path)
    original_data[model_version] = df
    all_data[model_version] = get_data(df)


In [48]:
features = ['Species_Humans', 'Age_Young', 'Fitness_Fit', 'Gender_Female',
       'SocialValue_High', 'Utilitarianism_More']

In [49]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt
# import seaborn as sns
# from functools import reduce
# from scipy.spatial.distance import pdist, squareform

# def prepare_data(all_data, features):
#     # Find common languages across all models
#     common_languages = list(reduce(set.intersection, (set(df.index) for df in all_data.values())))
#     print(f"Number of common languages: {len(common_languages)}")
#     model_matrices = {}
#     for model, df in all_data.items():
#         # Filter for common languages and select features
#         matrix = df.loc[common_languages, features].values
#         model_matrices[model] = matrix.flatten()
    
#     return pd.DataFrame(model_matrices).T

# def calculate_distances(data):
#     # Calculate pairwise distances between models
#     distances = pdist(data.values, metric='euclidean')
#     # Convert the condensed distance matrix to a square matrix
#     square_distances = squareform(distances)
#     return pd.DataFrame(square_distances, index=data.index, columns=data.index)

# def perform_pca(distances):
#     scaler = StandardScaler()
#     scaled_distances = scaler.fit_transform(distances)
#     pca = PCA(n_components=2)
#     return pca.fit_transform(scaled_distances)

# def plot_model_similarity(pca_result, model_names):
#     plt.figure(figsize=(12, 8))
#     sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1])
    
#     for i, model in enumerate(model_names):
#         plt.annotate(model, (pca_result[i, 0], pca_result[i, 1]), xytext=(5, 5), 
#                      textcoords='offset points', ha='left', va='bottom',
#                      bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
#                      arrowprops=dict(arrowstyle = '->', connectionstyle='arc3,rad=0'))
    
#     plt.title("Model Similarity based on Performance Across Languages and Criteria")
#     plt.xlabel("First Principal Component")
#     plt.ylabel("Second Principal Component")
#     plt.tight_layout()
#     plt.savefig("imgs/model_similarity.png")
#     plt.show()
   

# # Main execution
# features = ['Species_Humans', 'Age_Young', 'Fitness_Fit', 'Gender_Female', 'SocialValue_High', 'Utilitarianism_More']
# data = prepare_data(all_data, features)
# distances = calculate_distances(data)
# pca_result = perform_pca(distances)
# plot_model_similarity(pca_result, data.index)

In [50]:
population=pd.read_csv("../../data/human/proportions_population.csv")
country_metadata = pd.read_csv("../../data/human/country_metadata.csv")

def preprocess_counry(df):
    df = pd.melt(df,id_vars='criterion',value_name='estimate',var_name='Languages')
    df = df.replace("---", np.nan)
    df.estimate = df.estimate.astype(float)
    ## compute gpt4 preference by country
    country_lang=population.merge(df,on=['Languages'])
    country_lang['weighted_estimate']=(country_lang.percentage*country_lang.estimate)/100
    country_lang=country_lang.groupby(['Country','ISO3','criterion']).agg({'weighted_estimate':np.sum}).reset_index()


    # merge with country metadata
    country_lang=country_lang.merge(country_metadata[["ISO3", "Country", "Cluster"]],on='ISO3')
    country_lang
    return country_lang
country_lang = preprocess_counry(original_data[MISTRAL_7B])

/tmp/ipykernel_251323/3086117054.py:11: FutureWarning:

The provided callable <function sum at 0x72f587fc2200> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.



In [51]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Load required data
population = pd.read_csv("../../data/human/proportions_population.csv")
country_metadata = pd.read_csv("../../data/human/country_metadata.csv")

# Function to preprocess data
def preprocess_country(df):
    df = pd.melt(
        df, id_vars=["criterion", "model"], value_name="estimate", var_name="Languages"
    )
    df.estimate = df.estimate.astype(float)
    country_lang = population.merge(df, on=["Languages"])
    country_lang["weighted_estimate"] = (
        country_lang.percentage * country_lang.estimate
    ) / 100
    country_lang = (
        country_lang.groupby(["Country", "ISO3", "criterion", "model"])
        .agg({"weighted_estimate": "sum"})
        .reset_index()
    )
    country_lang = country_lang.merge(
        country_metadata[["ISO3", "Country", "Cluster"]], on="ISO3"
    )
    return country_lang


# Define the criteria and clusters
criteria = ['Species_Humans', 'Age_Young', 'Fitness_Fit', 'Gender_Female', 'SocialValue_High', 'Utilitarianism_More']
nice_labels = {
    'Species_Humans': 'Sparing humans',
    'Age_Young': 'Sparing the young',
    'Fitness_Fit': 'Sparing the fit',
    'Gender_Female': 'Sparing females',
    'SocialValue_High': 'Sparing higher status',
    'Utilitarianism_More': 'Sparing more'
}
criteria = [c+"_Bin_Choice" for c in criteria]
nice_labels = { c+"_Bin_Choice": v for c,v in nice_labels.items()}
clusters = {0: 'East', 1: 'West', 2: 'South'}


In [52]:
# Create and save a plot for each model
output_dir = "imgs/radar_clusters"
os.makedirs(output_dir, exist_ok=True)

In [53]:
# Process data for all models
all_model_data = {}
for model, data in original_data.items():
    all_model_data[model] = preprocess_country(data)

KeyError: "The following id_vars or value_vars are not present in the DataFrame: ['model']"

In [43]:
# Add human data
humans = pd.read_csv("../../data/human/human_preferences_by_lang_converted.csv")
humans_to_our = {"Age": "Age_Young_Bin_Choice", "Fitness": "Fitness_Fit_Bin_Choice", "Gender": "Gender_Female_Bin_Choice" , "Social Status": "SocialValue_High_Bin_Choice", "Species": "Species_Humans_Bin_Choice", "No. Characters": "Utilitarianism_More_Bin_Choice"}
humans["Label"] = humans["Label"].apply(lambda x: humans_to_our[x])
humans = humans.rename(columns={"Label": "criterion"})
humans["model"] = "Humans"
humans = preprocess_country(humans)
humans["family"] = "Humans" 

ValueError: could not convert string to float: 'Humans'

In [32]:
import plotly.graph_objs as go
import pandas as pd
import os

def calculate_diff_from_humans(data, human_data):
    # Calculate average scores across all criteria for each cluster and model
    model_cluster_averages = data[data['criterion'].isin(criteria)] \
        .groupby(['model', 'Cluster', 'criterion'])['weighted_estimate'].mean().unstack()
    
    # Calculate human averages
    human_averages = human_data[human_data['criterion'].isin(criteria)] \
        .groupby(['Cluster', 'criterion'])['weighted_estimate'].mean().unstack()
    
    # Calculate absolute differences from Humans
    diff_averages = model_cluster_averages.sub(human_averages).abs()
    
    # Calculate mean difference across all criteria
    mean_diff = diff_averages.mean(axis=1).unstack()
    
    return mean_diff

# Calculate differences from humans for all models
all_diff_averages = calculate_diff_from_humans(all_model_data, all_model_data[all_model_data['model'] == 'Humans'])

# Color palette for clusters
cluster_colors = {'East': '#1f77b4', 'West': '#ff7f0e', 'South': '#2ca02c'}

# Create separate plots for each model
for model in all_diff_averages.index:
    fig = go.Figure()
    
    for cluster in clusters.keys():
        fig.add_trace(
            go.Bar(
                x=[clusters[cluster]],
                y=[all_diff_averages.loc[model, cluster]],
                name=clusters[cluster],
                marker_color=cluster_colors[clusters[cluster]]
            )
        )
    
    # Update layout
    fig.update_layout(
        title=f"Average Absolute Difference from Humans Across All Criteria by Cluster for {model}",
        xaxis_title="Cluster",
        yaxis_title="Average Absolute Difference from Humans",
        height=500,
        width=700,
        bargap=0.15,
        bargroupgap=0.1,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    # Save the plot as a PNG file
    # output_file = os.path.join(output_dir, f"{model.replace('/', '_')}_diff_from_humans_histogram.png")
    # fig.write_image(output_file)
    
    # Optionally display the plot
    fig.show()

